# Lesson 05
# Peter Lorenz

## 0. Preliminaries

Import the required libraries:

In [1]:
import matplotlib as mpl
import numpy as np
import pandas as pd

Set global options:

In [2]:
# Display plots inline
%matplotlib inline

# Display multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Suppress scientific notation
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Declare utility functions:

## Read data
First we import the data set:

In [3]:
# Internet location of the data set
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"

# Download the data into a dataframe object
cancer_data = pd.read_csv(url)

# Display shape and initial data
cancer_data.shape
cancer_data.head()

(698, 11)

,1000025,5,1,1.1,1.2,2,1.3,3,1.4,1.5,2.1
0,1002945,5,4,4,5,7,10,3,2,1,2
1,1015425,3,1,1,1,2,2,3,1,1,2
2,1016277,6,8,8,1,3,4,3,7,1,2
3,1017023,4,1,1,3,2,1,3,1,1,2
4,1017122,8,10,10,8,7,10,9,7,1,4


## 1. Test both entropy and the gini coefficient
In this section we test entropy and the gini coefficient to answer the question which performs better and why.

## 2. Find best hyperparameter settings
In this section we find the best hyperparameter settings for entropy and the gini coefficient.

## 3. Visualize both models and see which feature is selected for each criterion
In this section we visualize both models and see which feature is selected for each criterion. Our goal is to determine whether the same feature is selected for both models and ascertain the reason behind this.

## 4. Determine the AUC for the best model you can achieve
In this section we determine the AUC for the best model we can achieve. We inquire as to the precision and recal values and as to which might be the one we want to maximize.

## 5. Implications of using this type of machine learning algorithm for breast cancer analysis
Here we examine the implications of using this type of machine learning algorithm for breast cancer analysis.